<h1 style="text-align: center">Database to JSON Export Pipeline</h1>

This notebook extracts data from the PostgreSQL database and converts it to JSON/GeoJSON files for the frontend application. The exported files are placed in `../frontend/public/data/` for static serving.

## AVAs

In [1]:
import os
import json
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
dotenv_path = '../.env'
load_dotenv(dotenv_path=dotenv_path)

DATABASE_URL = os.getenv('DATABASE_URL')
if not DATABASE_URL:
    raise ValueError("DATABASE_URL not found in .env file. Make sure .env file exists in the 'backend' directory.")

# Create database connection
engine = create_engine(DATABASE_URL)

# Define output directory
output_dir = Path('../../frontend/public/data/')
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Database connection configured")
print(f"Output directory: {output_dir.absolute()}")
print(f"Setup complete.")

Database connection configured
Output directory: /home/tabernater/projects/VitisVeritas/backend/notebooks/../../frontend/public/data
Setup complete.


In [ ]:
# Query to fetch all AVAs with their geometry
avas_query = """
SELECT 
    id,
    name,
    description,
    ST_AsGeoJSON(geom) as geometry,
    created_at,
    updated_at
FROM avas
ORDER BY name;
"""

print("Fetching AVAs from database...")

# Execute query and create GeoDataFrame
with engine.connect() as connection:
    df_avas = pd.read_sql(avas_query, connection)

print(f"Retrieved {len(df_avas)} AVAs from database")

# Convert to GeoDataFrame
from shapely.geometry import shape

# Parse the geometry from GeoJSON strings using shapely
geometries = df_avas['geometry'].apply(
    lambda x: shape(json.loads(x)) if x else None
)

# Create GeoDataFrame
gdf_avas = gpd.GeoDataFrame(
    df_avas.drop('geometry', axis=1),
    geometry=geometries,
    crs='EPSG:4326'
)

# Display basic info
print(f"AVAs data preview:")
print(f"   - Total AVAs: {len(gdf_avas)}")
print(f"   - Columns: {list(gdf_avas.columns)}")
print(f"   - CRS: {gdf_avas.crs}")
print(f"   - Geometry types: {gdf_avas.geometry.geom_type.value_counts().to_dict()}")

# Display first few rows
print(f"\nFirst 3 AVAs:")
for idx, row in gdf_avas.head(3).iterrows():
    print(f"   - {row['name']}: {len(row['description'] or 'No description')} chars description")

📥 Fetching AVAs from database...
✅ Retrieved 12 AVAs from database
📊 AVAs data preview:
   - Total AVAs: 12
   - Columns: ['id', 'name', 'description', 'created_at', 'updated_at', 'geometry']
   - CRS: EPSG:4326
   - Geometry types: {'MultiPolygon': 12}

📋 First 3 AVAs:
   - Chehalem Mountains: 14 chars description
   - Dundee Hills: 14 chars description
   - Eola-Amity Hills: 14 chars description


In [ ]:
# Export to GeoJSON file
avas_output_path = output_dir / 'avas.geojson'

print(f"Exporting AVAs to: {avas_output_path}")

# Export as GeoJSON
gdf_avas.to_file(avas_output_path, driver='GeoJSON')

# Verify the file was created and get file size
if avas_output_path.exists():
    file_size_mb = avas_output_path.stat().st_size / (1024 * 1024)
    print(f"Successfully exported AVAs!")
    print(f"   - File: {avas_output_path}")
    print(f"   - Size: {file_size_mb:.2f} MB")
    print(f"   - Features: {len(gdf_avas)}")
    
    # Read a snippet to verify structure
    with open(avas_output_path, 'r') as f:
        sample = f.read(200)
        print(f"   - Sample content: {sample}...")
else:
    print("Failed to create AVAs GeoJSON file")

💾 Exporting AVAs to: ../../frontend/public/data/avas.geojson
✅ Successfully exported AVAs!
   - File: ../../frontend/public/data/avas.geojson
   - Size: 3.48 MB
   - Features: 12
   - Sample content: {
"type": "FeatureCollection",
"name": "avas",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": [
{ "type": "Feature", "properties": { "id": 10, "name"...
✅ Successfully exported AVAs!
   - File: ../../frontend/public/data/avas.geojson
   - Size: 3.48 MB
   - Features: 12
   - Sample content: {
"type": "FeatureCollection",
"name": "avas",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": [
{ "type": "Feature", "properties": { "id": 10, "name"...


## Wineries